In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.tensorboard import SummaryWriter
import Model.mymodel as mymodel
import Model.trainer as trainer

In [2]:
set_frame = 50

epochs = 60
batch_sz = 10
checkpoint_frequency = 5
learning_rate = 0.0005
gamma = 0.8


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dl, val_dl, test_dl = trainer.load_data()

dataloaders = {
    'train' : train_dl,
    'val' : val_dl,
    'test' : test_dl
}



network = trainer.get_simple_conv_net()
network = network.to(DEVICE)

optim = SGD(network.parameters(), lr=learning_rate)
lr_sch = ExponentialLR(optim, gamma)

writer = SummaryWriter()
stopp = trainer.EarlyStopper(patience = 5, tolerance = 20)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 50, 640, 480]          62,550
              ReLU-2         [-1, 50, 640, 480]               0
         MaxPool2d-3         [-1, 50, 320, 240]               0
            Conv2d-4         [-1, 50, 320, 240]          62,550
              ReLU-5         [-1, 50, 320, 240]               0
         MaxPool2d-6         [-1, 50, 160, 120]               0
           Flatten-7               [-1, 960000]               0
            Linear-8                  [-1, 128]     122,880,128
              ReLU-9                  [-1, 128]               0
           Linear-10                  [-1, 256]          33,024
             ReLU-11                  [-1, 256]               0
           Linear-12                   [-1, 64]          16,448
             ReLU-13                   [-1, 64]               0
           Linear-14                   

In [3]:
writer = trainer.train_model(network,epochs,dataloaders,optim,lr_sch,writer,stopp,checkpoint_frequency)

Train epoch[1/60], MiniBatch[24/24], Loss: 1.94610, Acc: 13.79310, LR: 0.00050
Eval epoch[1/60], MiniBatch[4/4], Loss: 2.12335, Acc: 12.50000                
Train epoch[2/60], MiniBatch[24/24], Loss: 2.53817, Acc: 22.41379, LR: 0.00040
Eval epoch[2/60], MiniBatch[4/4], Loss: 2.27455, Acc: 18.75000                
Train epoch[3/60], MiniBatch[24/24], Loss: 2.28445, Acc: 35.34483, LR: 0.00032
Eval epoch[3/60], MiniBatch[4/4], Loss: 2.79173, Acc: 18.75000                
Train epoch[4/60], MiniBatch[24/24], Loss: 1.70572, Acc: 42.24138, LR: 0.00026
Eval epoch[4/60], MiniBatch[4/4], Loss: 1.09005, Acc: 18.75000                
Train epoch[5/60], MiniBatch[24/24], Loss: 1.93529, Acc: 49.13793, LR: 0.00020
Eval epoch[5/60], MiniBatch[4/4], Loss: 0.84750, Acc: 31.25000                
Train epoch[6/60], MiniBatch[24/24], Loss: 1.78582, Acc: 50.86207, LR: 0.00016
Eval epoch[6/60], MiniBatch[4/4], Loss: 2.63550, Acc: 31.25000                
Train epoch[7/60], MiniBatch[24/24], Loss: 1.03096, 